In [4]:
import pickle
import os


data = pickle.load(open('SynthesisNet/results/viz/top_1000/skeletons-top-1000-valid.pkl', 'rb'))
inds = [0,1,2,3,15,20,22,32,47,60,61,62,64,65,68,72,96,104,156,245,292,295,525,540,812,958,1107,2024,2252,6180]
sts = [list(data)[ind] for ind in inds]
smiles = []
for st in sts:
    for st in data[st]:
        assert len(st.reactions) <= 4
        smiles.append(st.root.smiles)

smiles[:10]

['CCNS(=O)(=O)c1cc([N+](=O)[O-])ccc1C1=NC(c2ccc(N=C=O)cc2F)=NN1',
 'COC(=O)c1ccc(NC(=O)Nc2ccc(C#N)nc2)c(O)c1',
 'Nc1c(Cl)cc(F)cc1C1=NC(c2cc([N+](=O)[O-])c(F)cc2F)=NN1',
 'CC(=O)N(c1nc(CNC(=S)Nc2cc(Br)cc(Cl)c2C(=O)O)cs1)c1c(C)cc(C)cc1Cl',
 'COC(=O)c1ccc(OCCNC(=O)CN=C=S)c(N)c1F',
 'CC(C)(C)c1ccc2c(c1)C(=O)NC(CN=C=S)=N2',
 'CC(C)Nc1ccc(Cl)cc1NS(=O)(=O)Cc1cc(F)ccc1[N+](=O)[O-]',
 'CCCn1c(-c2cc(Cl)cc(Cl)c2[N+](=O)[O-])nc2ccccc21',
 'COCCOc1cc(NS(=O)(=O)Cc2cc(F)ccc2[N+](=O)[O-])c(C(=O)O)cc1OC',
 'Cc1ccc(N=C=O)c(C2=NNC(Cc3ccc(N=C=S)cc3)=N2)c1']

In [7]:
len(smiles)
with open('SynthesisNet/data/assets/molecules/depth-5.txt', 'w+') as f:
    for smile in smiles:
        f.write(f"{smile}\n")


In [52]:
import matplotlib.pyplot as plt
import numpy as np
lines = open('SynthesisNet/output_mcmc_drd2_top_k=3_max_num_rxns=3.txt.txt').readlines()
all_scores = []
all_smiles = []
all_indices = []
for line in lines:
    index, smiles, score_history, smiles_history, index_history = line.split()
    all_scores.append(list(map(float, score_history.split(','))))
    all_smiles.append(smiles_history.split(','))
    all_indices.append(list(map(int, index_history.split(','))))
all_scores = np.array(all_scores)

In [54]:
# keys = []
# for line in lines:
#     keys.append([])
#     all_smiles = line.split()[-2].split(',')
#     all_indices = line.split()[-1].split(',')
#     for smiles, indices in zip(all_smiles, all_indices):
#         keys[-1].append((smiles, indices))
# [len(set(key)) for key in keys]
all_scores

array([], dtype=float64)

In [53]:
fig = plt.Figure()
ax = fig.add_subplot(1,1,1)
max_accum = np.maximum.accumulate(all_scores, axis=-1)
for k in [1,3,10,100]:
    all_scores_flatten = all_scores.flatten()
    max_top_k = all_scores_flatten[all_scores_flatten.argsort()[-k:]].mean(axis=-1)
    ax.axhline(max_top_k, label=f'Top {k}={max_top_k}', color='purple')

max_mean = max_accum.mean(axis=0)
max_std = max_accum.std(axis=0)
x = np.arange(len(max_mean))
ax.plot(x, max_mean, label='Mean', color='blue')
ax.fill_between(x, max_mean-max_std, max_mean+max_std, color='blue', alpha=0.2, label='Standard Deviation')

ax.legend()
fig.savefig("test.png")

/tmp/ipykernel_4150332/3426805752.py:6: RuntimeWarning: Mean of empty slice.
  max_top_k = all_scores_flatten[all_scores_flatten.argsort()[-k:]].mean(axis=-1)
/tmp/ipykernel_4150332/3426805752.py:9: RuntimeWarning: Mean of empty slice.
  max_mean = max_accum.mean(axis=0)


TypeError: object of type 'numpy.float64' has no len()

In [21]:
all_scores.shape

(10, 21)

In [41]:
from synnet.config import DELIM
lines = open('SynthesisNet/output_reconstruct_top_k=3_max_num_rxns=4_max_rxns=-1.txt').readlines()
recovered = {'targets': [], 'decoded': []}
unrecovered = {'targets': [], 'decoded': []}
scores = []
for line in lines:
    index, res, score = line.split(' ')
    target, decoded, index = res.split(DELIM)
    score = float(score)
    if score == 1.0:
        recovered['targets'].append(target)
        recovered['decoded'].append(decoded)
    else:
        unrecovered['targets'].append(target)
        unrecovered['decoded'].append(decoded)
    scores.append(score)


print(np.mean(scores), (np.array(scores)==1).mean())

FileNotFoundError: [Errno 2] No such file or directory: 'SynthesisNet/output_reconstruct_top_k=3_max_num_rxns=4_max_rxns=-1.txt'

In [3]:
from tdc import Evaluator

for metric in "KL_divergence FCD_Distance Novelty Validity Uniqueness".split():
    evaluator = Evaluator(name=metric)
    try:
        score_recovered = evaluator(recovered["targets"], recovered["decoded"])
        score_unrecovered = evaluator(unrecovered["targets"], unrecovered["decoded"])
        score_all = evaluator(recovered["targets"]+unrecovered['targets'], recovered["decoded"]+unrecovered['decoded'])
    except TypeError:
        # Some evaluators only take 1 input args, try that.
        score_recovered = evaluator(recovered["decoded"])
        score_unrecovered = evaluator(unrecovered["decoded"])
        score_all = evaluator(recovered["decoded"]+unrecovered['decoded'])
    except Exception as e:
        print(e)
        score_recovered, score_unrecovered = np.nan, np.nan

    print(f"Evaluation metric for {evaluator.name}:")
    print(f"    Recovered score: {score_recovered:.2f}")
    print(f"  Unrecovered score: {score_unrecovered:.2f}")
    print(f"  Total score: {score_all:.2f}")


Evaluation metric for kl_divergence:
    Recovered score: 1.00
  Unrecovered score: 0.89
  Total score: 0.96
Evaluation metric for fcd_distance:
    Recovered score: 0.00
  Unrecovered score: 1.79
  Total score: 0.46
Evaluation metric for novelty:
    Recovered score: 0.01
  Unrecovered score: 1.00
  Total score: 0.50
Evaluation metric for validity:
    Recovered score: 1.00
  Unrecovered score: 1.00
  Total score: 1.00
Evaluation metric for uniqueness:
    Recovered score: 1.00
  Unrecovered score: 1.00
  Total score: 1.00


'0 CC(C)(C)OC(=O)N1CC(CCl)(C2=NN(C3(C(=O)O)CCOC3c3ccc(F)cc3)N=N2)C1_____CC(C)(C)OC(=O)N1CC(CCl)(C2=NN(CC(=O)N3CCOC(c4ccc(F)cc4)C3)N=N2)C1_____1 0.573170731707317\n'

In [30]:
evaluator = Evaluator(name='FCD_Distance')
evaluator(recovered["decoded"], recovered["targets"]+unrecovered["targets"])

In [ ]:
evaluator = Evaluator(name='FCD_Distance')
evaluator(recovered["decoded"], recovered["targets"]+unrecovered["targets"])
evaluator = Evaluator(name='KL_divergence')
evaluator(recovered["decoded"], recovered["targets"]+unrecovered["targets"])